In [22]:
import numpy as np
import pandas as pd

In [23]:
df_all = pd.read_excel("master_file_final.xlsx")
site_team_machine = pd.read_excel("Sitegroup_machines.xlsx", sheet_name='Sheet2')

In [35]:
# E.g. Daily arrival 2020-01-09
#dfins = df_all[df_all['CreatedDate'] == '2020-01-09']
#df1 = dfins[['MRN','CreatedDate','SiteGroup','Sitegroup_y','Intent','Category_x','SimApptDuration','TltDose','TxFracs','TxApptDuration']]

# With 3-month warm up period
dfins = df_all[(df_all['CreatedDate'] >= '2018-01-01')&
               (df_all['CreatedDate'] <= '2020-12-01')]
dfins.sort_values(by=['CreatedDate','CreatedTime'],inplace=True)

df1 = dfins[['MRN','CreatedDate','CreatedTime','SiteGroup','Sitegroup_y','Intent1','Category_x','SimApptDuration','TxUnit','TltDose','TxFracs','TxApptDuration',
             'TxStartDt','WaitTime']]

df1.shape

/Users/fanjia/opt/anaconda3/lib/python3.8/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


(15518, 14)

In [36]:
df1

,MRN,CreatedDate,CreatedTime,SiteGroup,Sitegroup_y,Intent1,Category_x,SimApptDuration,TxUnit,TltDose,TxFracs,TxApptDuration,TxStartDt,WaitTime
13462,2626026,2018-01-01,1999-11-30 12:27:24,GENITOURINARY(GU),GENITOURINARY(GU),Radical,Standard 3,30,NA09,5400.0,27.0,30,2018-01-17,16
48357,4503480,2018-01-02,1999-11-30 06:45:10,BREAST,BREAST,Radical,Standard 3,30,EA05,5000.0,25.0,30,2018-01-16,14
46684,4437449,2018-01-02,1999-11-30 09:42:47,SARCOMA,SARCOMA,Radical,Standard 3,45,WA17,5040.0,28.0,20,2018-01-11,9
41357,4253349,2018-01-02,1999-11-30 12:04:18,GENITOURINARY(GU),GENITOURINARY(GU),Radical,Standard 3,15,SV02,6600.0,33.0,30,2018-01-16,14
45579,4399675,2018-01-02,1999-11-30 13:36:57,CNS,LUNG,Complex Palliation,Standard 3,45,NA10,2400.0,2.0,60,2018-01-22,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53872,4794354,2020-12-01,1999-11-30 14:38:30,ENDOCRINE,ENDOCRINE,Complex Palliation,Urgent 2,30,SA04,2000.0,5.0,30,2020-12-09,8
53891,4803621,2020-12-01,1999-11-30 15:20:05,PAEDIATRIC,PAEDIATRIC,Radical,Standard 3,30,NA12,5400.0,30.0,30,2020-12-28,27
45325,4390566,2020-12-01,1999-11-30 16:22:39,GENITOURINARY(GU),GENITOURINARY(GU),Radical,Standard 3,30,NA10,6000.0,20.0,30,2020-12-10,9
51977,4665382,2020-12-01,1999-11-30 17:11:32,LUNG,LUNG,Palliative,Urgent 2,30,EA08,400.0,1.0,30,2020-12-02,1


In [40]:
df_merged = df1.merge(site_team_machine, left_on=['SiteGroup'], right_on=['Sitegroup'])
df_merged.sort_values(by=['CreatedDate','CreatedTime'],inplace=True)

In [41]:
df_merged.columns

Index(['MRN', 'CreatedDate', 'CreatedTime', 'SiteGroup', 'Sitegroup_y',
       'Intent1', 'Category_x', 'SimApptDuration', 'TxUnit', 'TltDose',
       'TxFracs', 'TxApptDuration', 'TxStartDt', 'WaitTime', 'Sitegroup',
       'Team', 'CT', 'linacs'],
      dtype='object')

In [42]:
df_out=df_merged[['MRN','CreatedDate','SiteGroup','Intent1','Category_x','SimApptDuration',
                 'TltDose','TxFracs','TxApptDuration','TxUnit','Team','CT','linacs','TxStartDt','WaitTime']]
df_out=df_out[(df_out['Category_x']!='Planned Delay 3')&(df_out['Category_x']!='Emergency 1')]

In [43]:
# Assign priority weights:

# create a list of our conditions
conditions = [
    (df_out['Category_x'] == 'Urgent 2'),
    (df_out['Category_x'] == 'Standard 3')
    ]

# create a list of the values we want to assign for each condition
values = [10, 1]
val_target = [3, 10]

# create a new column and use np.select to assign values to it using our lists as arguments
df_out['priority'] = np.select(conditions, values)
df_out['targetWait'] = np.select(conditions, val_target)
df_out.shape

(11803, 17)

In [44]:
df_out['CreatedDay']=0
for column in df_out.columns:
    df_out[column].fillna(df_out[column].mode()[0], inplace=True)
df_out.head()

,MRN,CreatedDate,SiteGroup,Intent1,Category_x,SimApptDuration,TltDose,TxFracs,TxApptDuration,TxUnit,Team,CT,linacs,TxStartDt,WaitTime,priority,targetWait,CreatedDay
0,2626026,2018-01-01,GENITOURINARY(GU),Radical,Standard 3,30,5400.0,27.0,30,NA09,3,4,"2, 9, 10, 15, 14",2018-01-17,16,1,10,0
1719,4503480,2018-01-02,BREAST,Radical,Standard 3,30,5000.0,25.0,30,EA05,2,3,"5, 6, 7, 8, 11, 14",2018-01-16,14,1,10,0
4680,4437449,2018-01-02,SARCOMA,Radical,Standard 3,45,5040.0,28.0,20,WA17,4,2,"12, 16, 17, 14",2018-01-11,9,1,10,0
1,4253349,2018-01-02,GENITOURINARY(GU),Radical,Standard 3,15,6600.0,33.0,30,SV02,3,4,"2, 9, 10, 15, 14",2018-01-16,14,1,10,0
5477,4399675,2018-01-02,CNS,Complex Palliation,Standard 3,45,2400.0,2.0,60,NA10,4,2,"12, 16, 17, 14",2018-01-22,20,1,10,0


In [45]:
df_out['Intent1'].unique()

array(['Radical', 'Complex Palliation', 'Palliative'], dtype=object)

In [46]:
#df_out.to_excel("ins_2018_2020_part1.xlsx")
df_out.to_excel('ins_2018_2020_heuristics.xlsx')